In [1]:
from gensim.models import Doc2Vec
import multiprocessing, tempfile
import logging

logger = logging.getLogger('train_doc2vec_model')
logger.setLevel(logging.DEBUG)
sh = logging.StreamHandler()
sh.setFormatter(logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s'))
logger.addHandler(sh)

Parameters for Doc2Vec model

In [2]:
model_file = "/home/cds/Documents/Models/doc2vec_4top2vec.model"

min_count = 20
hs = 1
negative = 0
epochs = 40

workers = multiprocessing.cpu_count()

doc2vec_args = {"vector_size": 300,
                "min_count": min_count,
                "window": 15,
                "sample": 1e-5,
                "negative": negative,
                "hs": hs,
                "epochs": epochs,
                "dm": 0,
                "dbow_words": 1,
                "workers": workers}

Load TREC 1-8 dataset


In [3]:
from utils import parser
text_data = "/home/cds/Documents/Corpus/lemmatized_trec_all.dat"

_, docs = parser(text_data)

temp = tempfile.NamedTemporaryFile(mode='w+t')
temp.writelines(docs)

doc2vec_args["corpus_file"] = temp.name

Train the model with top2vec parameters

In [4]:
model = Doc2Vec(**doc2vec_args)

Save the model

In [5]:
model.save(model_file)

UMAP TEST

In [1]:
from umap import umap_

print(umap_._HAVE_PYNNDESCENT)

True


In [1]:
import umap
from gensim.models import Doc2Vec
# numba import njit, set_num_threads, get_num_threads

#set_num_threads(4)
#print(get_num_threads())

model_file = "/home/cds/Documents/Models/doc2vec_4top2vec.model"
model = Doc2Vec.load(model_file)

doc_vectors = model.docvecs.vectors_docs[:800000]
#print(umap_._HAVE_PYNNDESCENT)
umap_model = umap.UMAP(n_neighbors=10, n_components=5, low_memory=False, verbose=True,
                      metric='cosine').fit(doc_vectors)

UMAP(angular_rp_forest=True, metric='cosine', n_components=5, n_neighbors=10,
     verbose=True)
Construct fuzzy simplicial set
Mon Nov  2 13:21:51 2020 Finding Nearest Neighbors
Mon Nov  2 13:21:51 2020 Building RP forest with 50 trees
Mon Nov  2 13:22:20 2020 NN descent for 20 iterations
	 1  /  20
	 2  /  20
	 3  /  20
	 4  /  20
	 5  /  20
	 6  /  20
	 7  /  20
	Stopping threshold met -- exiting after 7 iterations
Mon Nov  2 13:23:04 2020 Finished Nearest Neighbor Search
Mon Nov  2 13:23:08 2020 Construct embedding
	completed  0  /  200 epochs
	completed  20  /  200 epochs
	completed  40  /  200 epochs
	completed  60  /  200 epochs
	completed  80  /  200 epochs
	completed  100  /  200 epochs
	completed  120  /  200 epochs
	completed  140  /  200 epochs
	completed  160  /  200 epochs
	completed  180  /  200 epochs
Mon Nov  2 13:32:16 2020 Finished embedding


HdbSCAn

In [2]:
import hdbscan
cluster = hdbscan.HDBSCAN(min_cluster_size=15, metric='euclidean',
                          cluster_selection_method='eom').fit(umap_model.embedding_)